In [1]:
import pandas as pd
import calc_factors
from calc_factors import dataset, financial_columns, calc_and_simulate_risk_factors
import datetime
import numpy as np

In [2]:
df_risks, dict_of_risk_simulations = calc_and_simulate_risk_factors(dt_end=datetime.date(2023, 12, 20),
                                                                    steps_forward=1, num_samples=1)

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error as mae

def get_fair_value(df_risks, forecast_factors, fin_inst=None, risk_factors=None):
    ''' Прогноз справедливой стоимости финансовых инструментов от значений риск факторов
    df_risks: pd.DataFrame - тренировочный датасет историей котировок фин инструментов и значений риск факторов
    forecast_factors: pd.DataFrame - сет риск факторов, для которых нужно сделать прогноз
    fin_inst: list - список финансовых иснструментов (если None, прогнрзируются все)
    risk_factors: list - список риск факторов (если None, используются все)
    
    Returns
    =======
    pd.DataFrame - датафрейм с прогнозами справедливой стоимости инструментов
    '''
    
    # инициализация риск-факторов и финансовых инструментов
    if risk_factors is None:
        risk_factors = [col for col in df_risks.columns if 'risk_factor' in col]
    if fin_inst is None:
        fin_inst = list(df_risks.drop(columns=['dt'] + risk_factors))
    
    result = dict()
    
    # для каждого инструмента обучаем модель и оцениваем справедливую стоимость на прогнозах факторов
    for inst in fin_inst:
        model = LinearRegression().fit(df_risks[risk_factors].to_numpy(), np.log(df_risks[inst].to_numpy()))
        pred = np.exp(model.predict(forecast_factors[risk_factors]))
        
        result[inst] = pred
        
        if inst in forecast_factors.columns:
            print(f'for {inst} mae is {mae(forecast_factors[inst].to_numpy(), pred)}')
    
    result = pd.DataFrame(result)
    result['dt'] = forecast_factors['dt'].reset_index(drop=True)
    
    return result[['dt'] + fin_inst]

In [4]:
get_fair_value(df_risks.iloc[:-15], df_risks.iloc[-15:])

for close_B_1 mae is 3.6851625635621987
for close_B_2 mae is 2.929529287616939
for close_B_3 mae is 3.134014506513305
for close_B_4 mae is 2.638798770988842
for close_B_5 mae is 2.3581360689920077
for close_AFKS mae is 0.5652232561658844
for close_AGRO mae is 24.649148702129818
for close_BRENT mae is 18.2396327990744
for close_EUR_RUB__TOD mae is 4.278043401698221
for close_GAZP mae is 2.5251334449243075
for close_IMOEX mae is 17.66227263913103
for close_LKOH mae is 67.46932323496767
for close_NLMK mae is 3.318310313290785
for close_NVTK mae is 10.54117683844626
for close_PLZL mae is 84.30202313198957
for close_ROSN mae is 3.1820545545487446
for close_RTKM mae is 2.2026774048795876
for close_RTSI mae is 4.925877074748026
for close_SBER mae is 13.696853696938772
for close_USD000000TOD mae is 1.2684262292140838
for %_0,25y mae is 2.5301369645459224
for %_0,5y mae is 1.455972943055471
for %_0,75y mae is 0.9674623435651867
for %_1y mae is 0.7683942655594213
for %_2y mae is 0.49863165093186

,dt,close_B_1,close_B_2,close_B_3,close_B_4,close_B_5,close_AFKS,close_AGRO,close_BRENT,close_EUR_RUB__TOD,...,%_7y,%_10y,%_15y,%_20y,%_30y,discounted_coupon_1,discounted_coupon_2,discounted_coupon_3,discounted_coupon_4,discounted_coupon_5
0,2023-11-29,721.099748,851.509223,923.938088,826.727698,842.914755,16.771223,1510.773151,103.361309,91.168877,...,11.665081,11.769993,11.862822,11.903085,11.936353,122.695952,114.689292,101.142091,26.802718,75.102533
1,2023-11-30,717.727463,849.018226,923.345452,824.084880,841.346570,16.115586,1471.866265,99.078097,91.761658,...,11.734845,11.844883,11.943227,11.987793,12.026765,120.479613,111.909132,98.800668,27.172965,72.924302
2,2023-12-01,714.199713,845.624646,920.155090,819.939795,837.547407,15.698991,1430.871603,95.727014,93.145859,...,11.829915,11.928203,12.011115,12.045396,12.072242,118.304966,109.598944,96.633501,28.194727,71.394732
3,2023-12-04,708.677186,840.512625,917.799650,814.691137,834.025235,15.432065,1364.111640,93.872864,94.065801,...,11.999280,12.105301,12.184911,12.215672,12.239431,111.197977,102.641750,91.107445,29.009550,66.431981
4,2023-12-05,707.421164,838.987626,916.446526,812.554646,831.834211,15.529807,1351.305434,92.432149,96.178070,...,12.031693,12.127562,12.195526,12.220752,12.241024,113.042436,101.177145,89.859260,29.757297,65.053507
5,2023-12-06,706.862673,838.846347,916.211623,812.652332,831.808354,15.007194,1330.998852,91.399546,95.377924,...,12.038829,12.133492,12.202356,12.227235,12.245234,112.027114,100.710612,89.797105,29.472072,64.981010
6,2023-12-07,707.518942,839.890497,916.678226,812.679982,831.741483,14.912089,1300.062971,90.537143,95.426010,...,12.020735,12.123483,12.197210,12.225697,12.249835,107.076189,97.143198,86.317050,30.859247,62.976785
7,2023-12-08,706.776609,839.361499,918.102778,812.342219,832.407097,15.096167,1260.375375,90.901056,96.239153,...,12.047867,12.175794,12.263223,12.298175,12.331810,101.849657,91.665401,81.719104,31.372429,59.358512
8,2023-12-11,703.989722,837.131292,917.109371,810.091599,831.316743,14.786463,1167.707257,91.337287,95.664506,...,12.180824,12.319332,12.404366,12.434618,12.462416,89.743448,82.091623,73.566981,32.295592,53.492432
9,2023-12-12,700.884309,834.006232,914.333064,807.314870,828.454453,14.452925,1226.163317,93.281089,93.179245,...,12.247110,12.364365,12.439285,12.467126,12.492638,96.601845,89.445365,80.266124,31.641809,58.085563
